## Build a Wflow sediment model

This notebook demonstrates how to prepare a **Wflow sediment** model using the command line interace (CLI).

All lines in this notebook which start with `!` are executed from the command line. Within the notebook environment the logging messages are shown after completion. You can also copy these lines and paste these in your shell to get more direct feedback.

### hydroMT CLI build interface

Lets first check if the Wflow sediment model is recognized by hydromt

In [1]:
# this should return "wflow, wflow_sediment"
!hydromt --models

hydroMT model plugins: wflow (v0.1.3.dev0), wflow_sediment (v0.1.3.dev0)


Using the **hydromt build** API we can setup a complete model from scratch. Let's get an overview of all the available options:

In [2]:
!hydromt build --help

Usage: hydromt build [OPTIONS] MODEL MODEL_ROOT REGION

  Build models from source data.

  Example usage:
  --------------

  To build a wflow model for a subbasin using and point coordinates snapped to cells with stream order >= 4
  hydromt build wflow /path/to/model_root "{'subbasin': [-7.24, 62.09], 'strord': 4}" -i /path/to/wflow_config.ini

  To build a wflow model based on basin ID
  hydromt build wflow /path/to/model_root "{'basin': 230001006}"

  To build a sfincs model based on a bbox (for Texel)
  hydromt build sfincs /path/to/model_root "{'bbox': [4.6891,52.9750,4.9576,53.1994]}"

Options:
  -r, --res FLOAT             Model resolution in model src.
  --build-base / --build-all  Deprecated!
  --opt TEXT                  Component specific keyword arguments, see the
                              setup_<component> method of the specific model
                              for more information about the arguments.

  -i, --config PATH           Path to hydroMT configuration fi

### Model setup configuration

The ini-file contains the model setup configuration and determines which components are build and in which order and optionally sets non-default arguments for each component. This configuration is passed to hydromt using `-i <path_to_ini_file>`. We have prepared several example ini-files which are available in the model repository [examples folder](https://github.com/Deltares/hydromt_wflow/tree/main/examples) and from the [docs(build_configuration)](https://deltares.github.io/hydromt_wflow/latest/user_guide/wflow/build_configuration.html). 

Each header as shown between `[...]` (e.g. `[setup_basemaps]`) corresponds to a model component. All model components are explained in the [docs(model_components)](https://deltares.github.io/hydromt_wflow/latest/user_guide/wflow/components.html). 

We will load the default wflow sediment build ini file for inspection:

In [3]:
fn_ini = 'wflow_sediment_build.ini'
with open(fn_ini, 'r') as f:
    txt = f.read()
print(txt)

[global]
data_libs        = []               # add optional paths to data yml files

[setup_config]                      # options parsed to wflow ini file <section>.<option>
starttime = 2010-01-01T00:00:00
endtime = 2010-03-31T00:00:00
timestepsecs = 86400

[setup_basemaps]
hydrography_fn   = merit_hydro      # source hydrography data {merit_hydro, merit_hydro_1k}
basin_index_fn   = merit_hydro_index # source of basin index corresponding to hydrography_fn
upscale_method   = ihu              # upscaling method for flow direction data, by default 'ihu'

[setup_rivers]
hydrography_fn   = merit_hydro   # source hydrography data {merit_hydro, merit_hydro_1k}
river_upa        = 30               # minimum upstream area threshold for the river map [km2]
slope_len        = 2000             # length over which tp calculate river slope [m]

[setup_reservoirs]
reservoirs_fn    = hydro_reservoirs # source for reservoirs based on GRAND: {hydro_reservoirs}; None to skip
min_area         = 1.0       

### Setup wflow sediment model from scratch

In [4]:
# NOTE: copy this line (without !) to your shell for more direct feedback
!hydromt build wflow_sediment "./wflow_test_sediment" "{'subbasin': [12.2051, 45.8331], 'strord': 4, 'bounds': [11.70, 45.35, 12.95, 46.70]}" -r 0.00833 -i wflow_sediment_build.ini -vv

2021-09-01 15:48:14,911 - build - log - DEBUG - Writing log messages to new file /home/runner/work/hydromt_wflow/hydromt_wflow/docs/examples/examples/wflow_test_sediment/hydromt.log.
2021-09-01 15:48:14,911 - build - log - INFO - HydroMT version: 0.4.3.dev
2021-09-01 15:48:14,911 - build - main - INFO - Building instance of wflow_sediment model at /home/runner/work/hydromt_wflow/hydromt_wflow/docs/examples/examples/wflow_test_sediment.
2021-09-01 15:48:14,911 - build - main - INFO - User settings:


2021-09-01 15:48:14,966 - build - model_plugins - DEBUG - Loaded model plugin 'wflow_sediment = hydromt_wflow.wflow_sediment.WflowSedimentModel' (0.1.3.dev0)
2021-09-01 15:48:14,966 - build - model_api - INFO - Initializing wflow_sediment model from hydromt_wflow (v0.1.3.dev0).
2021-09-01 15:48:14,967 - build - model_api - DEBUG - Setting model config options.
2021-09-01 15:48:14,969 - build - model_api - DEBUG - Default config read from /usr/share/miniconda/envs/hydromt-wflow/lib/python3.9/site-packages/hydromt_wflow/data/wflow_sediment/wflow_sediment.toml
2021-09-01 15:48:14,970 - build - model_api - INFO - setup_basemaps.region: {'subbasin': [12.2051, 45.8331], 'strord': 4, 'bounds': [11.7, 45.35, 12.95, 46.7]}
2021-09-01 15:48:14,970 - build - model_api - INFO - setup_basemaps.res: 0.00833
2021-09-01 15:48:14,970 - build - model_api - INFO - setup_basemaps.hydrography_fn: merit_hydro
2021-09-01 15:48:14,970 - build - model_api - INFO - setup_basemaps.basin_index_fn: merit_hydro_ind

2021-09-01 15:48:15,063 - build - data_adapter - INFO - DataCatalog: Getting merit_hydro RasterDataset raster data from /home/runner/.hydromt_data/data/v0.0.4/merit_hydro/{variable}.tif


2021-09-01 15:48:15,177 - build - basin_mask - DEBUG - Parsed region (kind=subbasin): {'strord': 4, 'bounds': [11.7, 45.35, 12.95, 46.7], 'xy': [12.2051, 45.8331]}


2021-09-01 15:48:19,203 - build - basin_mask - INFO - basin bbox: [11.7750, 45.8042, 12.7450, 46.6900] / size: 664756


...

2021-09-01 15:48:48,458 - build - data_adapter - DEBUG - RasterDataset: Convert units for 21 variables.


2021-09-01 15:48:49,911 - build - model_api - INFO - setup_gauges.gauges_fn: grdc
2021-09-01 15:48:49,911 - build - model_api - INFO - setup_gauges.source_gdf: None
2021-09-01 15:48:49,911 - build - model_api - INFO - setup_gauges.snap_to_river: True
2021-09-01 15:48:49,911 - build - model_api - INFO - setup_gauges.mask: None
2021-09-01 15:48:49,911 - build - model_api - INFO - setup_gauges.derive_subcatch: False
2021-09-01 15:48:49,911 - build - model_api - INFO - setup_gauges.derive_outlet: True
2021-09-01 15:48:49,911 - build - model_api - INFO - setup_gauges.basename: None
2021-09-01 15:48:49,911 - build - model_api - INFO - setup_gauges.update_toml: True
2021-09-01 15:48:49,912 - build - model_api - INFO - setup_gauges.gauge_toml_header: None
2021-09-01 15:48:49,912 - build - model_api - INFO - setup_gauges.gauge_toml_param: None
2021-09-01 15:48:49,912 - build - wflow - INFO - Gauges locations set based on river outlets.


2021-09-01 15:48:49,929 - build - wflow - INFO - Gauges map based on catchment river outlets added.
2021-09-01 15:48:49,929 - build - data_adapter - INFO - DataCatalog: Getting grdc GeoDataFrame csv data from /home/runner/.hydromt_data/data/v0.0.4/grdc.csv
2021-09-01 15:48:49,934 - build - data_adapter - INFO - GeoDataFrame: Read csv data and clip to geom (epsg:4326) [11.775, 45.804, 12.745, 46.690].
2021-09-01 15:48:49,958 - build - wflow - INFO - 3 grdc gauge locations found within domain
2021-09-01 15:48:49,974 - build - wflow - INFO - Gauges map from grdc added.


2021-09-01 15:48:49,996 - build - wflow - INFO - Write model data to /home/runner/work/hydromt_wflow/hydromt_wflow/docs/examples/examples/wflow_test_sediment
2021-09-01 15:48:49,996 - build - model_api - INFO - Writing model config to /home/runner/work/hydromt_wflow/hydromt_wflow/docs/examples/examples/wflow_test_sediment/wflow_sediment.toml
2021-09-01 15:48:49,996 - build - wflow - INFO - Write staticmaps to /home/runner/work/hydromt_wflow/hydromt_wflow/docs/examples/examples/wflow_test_sediment/staticmaps.nc


2021-09-01 15:48:50,462 - build - wflow - INFO - Writing model staticgeom to file.


The example above means the following: run **hydromt build** with:

* `wflow_sediment` : i.e. build a wflow sediment model
* `./wflow_test_sediment` : output model folder
* `"{'subbasin': [12.2051, 45.8331], 'strord': 4, 'bounds': [11.70, 45.35, 12.95, 46.70]}"` : derive a subbasin with its outlet at the given x,y coordinates and within the given bounds [xmin, ymin, xmax, ymax] (WGS84) snapped to a river with minimum stream order (strord) of 4. All *REGION* options are described in the [docs](https://deltares.github.io/hydromt/latest/user_guide/cli.html#region-options)
* `-r 0.0083333` : build the model at a 30 arc sec (~1km) resolution
* `-i wflow_sediment_build.ini` : setup configuration file using
* `-vv` : give some extra verbosity (2 * v) to display feedback on screen. Now debug messages are provided.


Next we check which files have been created. The model root should contain two netcdf (.nc) files, one for the staticmaps and one for the forcing data, a wflow configuration (.toml) file. These files are sufficient to run the wflow sediment model. In addition several geometry (.geojson) files are saved in the *staticgeoms* folder.  Finally, the setup process is logged in the hydromt.log file. 

In [5]:
import os
root = 'wflow_test_sediment'
for path, _, files in os.walk(root):
    print(path)
    for name in files:
        if name.endswith('.xml'):
            continue
        print(f' - {name}')

wflow_test_sediment
 - wflow_sediment.toml
 - hydromt.log
 - staticmaps.nc
wflow_test_sediment/setup
wflow_test_sediment/inmaps
wflow_test_sediment/staticmaps
wflow_test_sediment/run_default
wflow_test_sediment/staticgeoms
 - gauges.geojson
 - basins.geojson
 - gauges_grdc.geojson
 - lakes.geojson
 - rivers.geojson
 - region.geojson
wflow_test_sediment/instate


### Visualize and/or inspect model schematization

You can (copy and) adapt the following example notebooks in order to visualize your Wflow sediment model: 

* The **wflow plot** example notebook contains scripts to visualize your model
* The **wflow nc to raster files** example notebook contains scripts to write the nc model files to a netcdf which can be used to inspect (and modify) the model layers in e.g. QGIS.

### Add Wflow sediment layers to an existing Wflow model

If you already have a hydrological **Wflow** model and you want to extend it in order to include **sediment** as well, then you do not need to build the wflow_sediment from scratch. You can instead **update** the wflow model with the additional components needed by wflow_sediment. 

These components are available in the template **wflow_extend_sediment.ini**:

In [6]:
fn_ini = 'wflow_extend_sediment.ini'
with open(fn_ini, 'r') as f:
    txt = f.read()
print(txt)

[global]
data_libs        = []               # add optional paths to data yml files

[setup_config]
# model.dolake = True               # uncomment if there are lakes in the wflow model
# model.doreservoir = True          # uncomment if there are reservoirs in the wflow model

[setup_riverbedsed]
bedsed_mapping_fn   = None             # path to a mapping csv file from streamorder to river bed particles characteristics

[setup_lulcmaps]
lulc_fn          = globcover        # source for lulc maps: {globcover, vito, corine}

[setup_canopymaps]
canopy_fn        = simard           # source for vegetation canopy height: {simard}

[setup_soilmaps]
soil_fn          = soilgrids        # source for soilmaps: {soilgrids}
usleK_method     = renard           # method to compute the USLE K factor: {renard, epic}

[setup_gauges]
gauges_fn        = grdc             # If not None add gaugemap. Either a path or known gauges_fn: {grdc}
snap_to_river    = True             # If True snaps gauges from source

Let's update the hydrological **Wflow** model available from our example for the Piave subbasin:

In [7]:
# NOTE: copy this line (without !) to your shell for more direct feedback
!hydromt update wflow_sediment "wflow_piave_subbasin" -o "./wflow_test_extend_sediment" -i wflow_extend_sediment.ini -vv

2021-09-01 15:48:54,084 - update - log - DEBUG - Writing log messages to new file /home/runner/work/hydromt_wflow/hydromt_wflow/docs/examples/examples/wflow_test_extend_sediment/hydromt.log.
2021-09-01 15:48:54,084 - update - log - INFO - HydroMT version: 0.4.3.dev
2021-09-01 15:48:54,084 - update - main - INFO - Updating wflow_sediment model at /home/runner/work/hydromt_wflow/hydromt_wflow/docs/examples/examples/wflow_piave_subbasin (r).
2021-09-01 15:48:54,084 - update - main - INFO - Output dir: /home/runner/work/hydromt_wflow/hydromt_wflow/docs/examples/examples/wflow_test_extend_sediment
2021-09-01 15:48:54,084 - update - main - INFO - User settings:


2021-09-01 15:48:54,129 - update - model_plugins - DEBUG - Loaded model plugin 'wflow_sediment = hydromt_wflow.wflow_sediment.WflowSedimentModel' (0.1.3.dev0)
2021-09-01 15:48:54,130 - update - model_api - INFO - Initializing wflow_sediment model from hydromt_wflow (v0.1.3.dev0).
2021-09-01 15:48:54,130 - update - model_api - ERROR - Model config file not found at /home/runner/work/hydromt_wflow/hydromt_wflow/docs/examples/examples/wflow_piave_subbasin/wflow_sediment.toml
2021-09-01 15:48:54,130 - update - model_api - ERROR - Model config file not found at /home/runner/work/hydromt_wflow/hydromt_wflow/docs/examples/examples/wflow_piave_subbasin/wflow_sediment.toml
2021-09-01 15:48:54,130 - update - wflow - INFO - Read staticmaps from /home/runner/work/hydromt_wflow/hydromt_wflow/docs/examples/examples/wflow_piave_subbasin/staticmaps.nc


2021-09-01 15:48:54,214 - update - wflow - INFO - Reading model intbl files.
2021-09-01 15:48:54,214 - update - model_api - ERROR - Model config file not found at /home/runner/work/hydromt_wflow/hydromt_wflow/docs/examples/examples/wflow_piave_subbasin/wflow_sediment.toml
2021-09-01 15:48:54,215 - update - wflow - INFO - Reading model staticgeom files.


2021-09-01 15:48:54,406 - update - model_api - ERROR - Model config file not found at /home/runner/work/hydromt_wflow/hydromt_wflow/docs/examples/examples/wflow_piave_subbasin/wflow_sediment.toml
2021-09-01 15:48:54,406 - update - wflow - INFO - Read forcing from /home/runner/work/hydromt_wflow/hydromt_wflow/docs/examples/examples/wflow_piave_subbasin/inmaps.nc


2021-09-01 15:48:54,419 - update - wflow - INFO - Model read
2021-09-01 15:48:54,431 - update - model_api - INFO - setup_riverbedsed.bedsed_mapping_fn: None
2021-09-01 15:48:54,431 - update - wflow_sediment - INFO - Preparing riverbedsed parameter maps.
2021-09-01 15:48:54,447 - update - landuse - INFO - Deriving D50_River using average resampling (nodata=-999.0).


...

2021-09-01 15:48:58,485 - update - wflow - INFO - Gauges map based on catchment river outlets added.
2021-09-01 15:48:58,485 - update - data_adapter - INFO - DataCatalog: Getting grdc GeoDataFrame csv data from /home/runner/.hydromt_data/data/v0.0.4/grdc.csv
2021-09-01 15:48:58,489 - update - data_adapter - INFO - GeoDataFrame: Read csv data and clip to geom (epsg:4326) [11.775, 45.804, 12.745, 46.690].
2021-09-01 15:48:58,515 - update - wflow - INFO - 3 grdc gauge locations found within domain


2021-09-01 15:48:59,439 - update - model_api - DEBUG - Default config read from /usr/share/miniconda/envs/hydromt-wflow/lib/python3.9/site-packages/hydromt_wflow/data/wflow_sediment/wflow_sediment.toml
2021-09-01 15:48:59,439 - update - wflow - INFO - Gauges map from grdc added.
2021-09-01 15:48:59,478 - update - wflow - INFO - Write model data to /home/runner/work/hydromt_wflow/hydromt_wflow/docs/examples/examples/wflow_test_extend_sediment
2021-09-01 15:48:59,478 - update - model_api - INFO - Writing model config to /home/runner/work/hydromt_wflow/hydromt_wflow/docs/examples/examples/wflow_test_extend_sediment/wflow_sediment.toml
2021-09-01 15:48:59,479 - update - wflow - INFO - Write staticmaps to /home/runner/work/hydromt_wflow/hydromt_wflow/docs/examples/examples/wflow_test_extend_sediment/staticmaps.nc


2021-09-01 15:49:00,433 - update - wflow - INFO - Writing model staticgeom to file.


2021-09-01 15:49:00,664 - update - wflow - INFO - Write forcing file
2021-09-01 15:49:00,682 - update - wflow - INFO - Process forcing; saving to /home/runner/work/hydromt_wflow/hydromt_wflow/docs/examples/examples/wflow_test_extend_sediment/inmaps.nc
[                                        ] | 0% Completed |  0.0s

[########################################] | 100% Completed |  0.1s


The example above means the following: run **hydromt update** with:

* `wflow_sediment` : i.e. update a wflow_sediment model (in our case a wflow model but with wflow_sediment components)
* `wflow_piave_subbasin` : hydrological wflow model folder
* `-o "./wflow_test_extend_sediment"`: output combined wflow hydrology+sediment models
* `-i wflow_extend_sediment.ini` : setup configuration file containing wflow sediment specific components
* `-vv` : give some extra verbosity (2 * v) to display feedback on screen. Now debug messages are provided.